In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

tile_shadow_path = "../data/data_single_tile/shadow_index/paul_linke_shadow.csv"
monthly_df = pd.read_csv(tile_shadow_path, index_col=0)
monthly_df.index.name = "tree_id"
monthly_df

In [ ]:
all_shadow_path = "../data/shadow_index/berlin_shadow_box_220323.csv"
all_df = pd.read_csv(all_shadow_path, index_col=0)
all_df.index.name = "tree_id"
all_df

In [ ]:
s1 = pd.merge(monthly_df, all_df, how='inner', on=['tree_id'])
s1

In [ ]:
s2 = s1.rename(columns={"winter": "355", "spring": "80", "summer": "172", "autumn": "266"})

In [ ]:
seasonal_df = s2[['355', '80', "172", "266"]]
seasonal_df

In [ ]:
seasonal_df = seasonal_df.T
monthly_df = monthly_df.T

In [ ]:
ax = monthly_df.plot(marker='.', figsize=(10,7), title="Shadow index per season", ylabel= "shadow index", xlabel = "days", grid=True)

In [ ]:
ax = seasonal_df.plot(marker='.', figsize=(10,7), title="Shadow index per season", ylabel= "shadow index", xlabel = "days", grid=True)

In [ ]:
s2 = s2.rename(columns={"31": "031", "61": "061", "80": "080", "91": "091"})
merged_df = s2.reindex(sorted(s2.columns), axis=1)
merged_df 

In [ ]:
merged_df = merged_df.T

In [ ]:
ax = merged_df .plot(marker='.', figsize=(10,7), title="Shadow index per season", ylabel= "shadow index", xlabel = "days", grid=True)

In [ ]:
ax = monthly_df.plot(marker='.', figsize=(10,7), title="Shadow index per season", ylabel= "shadow index", xlabel = "days", grid=True)